In [ ]:
import pandas as pd
import os
year = 2020
yeara = str(year)
year2 = year - 1
yearb = str(year2)


## https://www.fec.gov/data/browse-data/?tab=bulk-data

## Load contribution Data

In [ ]:
# Step 2: Read the header file into a DataFrame
df = pd.read_csv('indiv_header_file.csv')

# Step 3: Inspect the DataFrame
print(df.head())

# Step 4: Save the DataFrame to a file
column_names = df.columns.tolist()

#df1 = pd.read_csv('20.txt', sep='|', names=column_names)

try:
    df1 = pd.read_csv('16.txt', sep='|', engine='python', error_bad_lines=False)
except Exception as e:
    print(f"Error reading CSV file: {e}")

Empty DataFrame
Columns: [CMTE_ID, AMNDT_IND, RPT_TP, TRANSACTION_PGI, IMAGE_NUM, TRANSACTION_TP, ENTITY_TP, NAME, CITY, STATE, ZIP_CODE, EMPLOYER, OCCUPATION, TRANSACTION_DT, TRANSACTION_AMT, OTHER_ID, TRAN_ID, FILE_NUM, MEMO_CD, MEMO_TEXT, SUB_ID]
Index: []

[0 rows x 21 columns]


C:\Users\Joel\AppData\Local\Temp\ipykernel_9744\466088873.py:13: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df1 = pd.read_csv('20.txt', sep='|', engine='python', error_bad_lines=False)
Skipping line 319331: '|' expected after '"'
Skipping line 1986154: '|' expected after '"'
Skipping line 3438882: '|' expected after '"'
Skipping line 3544071: '|' expected after '"'
Skipping line 3651830: '|' expected after '"'
Skipping line 6539806: '|' expected after '"'
Skipping line 6548428: '|' expected after '"'
Skipping line 7472313: field larger than field limit (131072)
Skipping line 7563227: field larger than field limit (131072)
Skipping line 7564267: field larger than field limit (131072)
Skipping line 7762689: '|' expected after '"'
Skipping line 9441472: '|' expected after '"'
Skipping line 9898873: '|' expected after '"'
Skipping line 9953625: '|' expected after '"'
Skipping line 10056787: '|

In [ ]:
df1.shape[0]

## Filter out irrelevant contribution data

In [2]:
can_ids = pd.read_excel('filtered_project_committees.xlsx')
can_ids_list = can_ids['committee_id'].to_list()
len(can_ids_list)

In [4]:
filtered_df1 = df1[df1['CMTE_ID'].isin(can_ids_list)]

## Drop irrelevant columns and format date columns

In [ ]:
selected_columns = filtered_df1[['CMTE_ID', 'TRANSACTION_DT', 'TRANSACTION_AMT']]
selected_columns['TRANSACTION_DT'] = selected_columns['TRANSACTION_DT'].astype(str)

# remove rows with out properly formated dates
selected_columns = selected_columns[selected_columns['TRANSACTION_DT'].apply(len).isin([9, 10])]

# formate date column
selected_columns['Year'] = selected_columns['TRANSACTION_DT'].str[-6:-2]
selected_columns['Date'] = selected_columns['TRANSACTION_DT'].str[-8:-6]
selected_columns['Month'] = selected_columns['TRANSACTION_DT'].str[:-8]

# remove rows that arnt within two years of the election
selected_columns = selected_columns[selected_columns['Year'].str.startswith((yeara, yearb))]
selected_columns.shape[0]

## Verify Data Quality

In [ ]:
unique_combos = selected_columns[['Month', 'Year']]

# Group by 'Month' and 'Year', then get the counts
combo_counts = unique_combos.groupby(['Month', 'Year']).size().reset_index(name='Count')
combo_counts = combo_counts.apply(pd.to_numeric)
combo_counts = combo_counts.sort_values(by=['Year', 'Month'])
print(combo_counts)

## Create and download an excel file for each month of contributions

In [ ]:
# create 'months' folder if it doesn't exist
if not os.path.exists('months'):
    os.makedirs('months')

In [ ]:
# list of all months
unique_year_month_combos = selected_columns[['Year', 'Month']].drop_duplicates()
year_month_list = [(row['Year'], row['Month']) for _, row in unique_year_month_combos.iterrows()]

# iterate over each month
for year, month in year_month_list:

    filtered_df = selected_columns[(selected_columns['Year'] == year) & (selected_columns['Month'] == month)]

    # save to the folder
    file_path = os.path.join('months/', f'{year}_{month}.csv')
    filtered_df.to_csv(file_path, index=False)

print("Data frames have been saved successfully.")

# 2020

## Above worked for 16 and 18 but the file size for 2020 was too big so i had to use a program called H-J split to break it down into 14 smaller files

## of the 14 files 3 of them did not have '.0' at the end of the date. so i had to adust the code to make it work for them. (4 &10)

In [ ]:
number = '008'

import pandas as pd
# Step 2: Read the header file into a DataFrame
df = pd.read_csv('indiv_header_file.csv')

# Step 3: Inspect the DataFrame
print(df.head())

# Step 4: Save the DataFrame to a file
column_names = df.columns.tolist()

Empty DataFrame
Columns: [CMTE_ID, AMNDT_IND, RPT_TP, TRANSACTION_PGI, IMAGE_NUM, TRANSACTION_TP, ENTITY_TP, NAME, CITY, STATE, ZIP_CODE, EMPLOYER, OCCUPATION, TRANSACTION_DT, TRANSACTION_AMT, OTHER_ID, TRAN_ID, FILE_NUM, MEMO_CD, MEMO_TEXT, SUB_ID]
Index: []

[0 rows x 21 columns]


In [ ]:
#df1 = pd.read_csv('20.txt', sep='|', names=column_names)
df1 = pd.read_csv(f'chunks/20.txt.{number}.txt', sep='|', engine='python', names=column_names, error_bad_lines=False)


C:\Users\Joel\AppData\Local\Temp\ipykernel_14680\2104414943.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df1 = pd.read_csv(f'chunks/20.txt.{number}.txt', sep='|', engine='python', names=column_names, error_bad_lines=False)


In [ ]:
df1.shape[0]
df1.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,...,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
0,GINEER,05232019,250,C00401224,857500,1369136,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,4011320201684008049,None,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
1,C00697441,A,Q2,P2020,202001089167037289,15E,IND,"FREEMON, PHILLIP",SAN FRANCISCO,CA,...,SELF-EMPLOYED,ATTORNEY,4162019.0,1000.0,C00401224,747383,1369136.0,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,4.011320e+18
2,C00697441,A,Q2,P2020,202001089167037290,15E,IND,"FREEMON, PHILLIP",SAN FRANCISCO,CA,...,SELF-EMPLOYED,ATTORNEY,5292019.0,1000.0,C00401224,867438,1369136.0,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,4.011320e+18
3,C00697441,A,Q2,P2020,202001089167043820,15E,IND,"LOCKHART, ARTHUR",SAN RAMON,CA,...,MVI HEALTH INC,EXECUTIVE,4202019.0,250.0,C00401224,769500,1369136.0,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,4.011320e+18
4,C00697441,A,Q2,P2020,202001089167043821,15E,IND,"LOCKHART, ARTHUR",SAN RAMON,CA,...,MVI HEALTH INC,EXECUTIVE,5202019.0,250.0,C00401224,846695,1369136.0,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,4.011320e+18


## Filter out irrelevant contribution data

In [ ]:
can_ids = pd.read_excel('filtered_project_committees.xlsx')
can_ids_list = can_ids['committee_id'].to_list()

In [ ]:
len(can_ids_list)

265

In [ ]:
filtered_df1 = df1[df1['CMTE_ID'].isin(can_ids_list)]
#filtered_df1.to_csv('test2_year_16.csv', index=False)
filtered_df1.shape[0]

402720

## Drop irrelevant columns and format date columns

In [ ]:
selected_columns = filtered_df1[['CMTE_ID', 'TRANSACTION_DT', 'TRANSACTION_AMT']]
selected_columns['TRANSACTION_DT'] = selected_columns['TRANSACTION_DT'].astype(str)
selected_columns.to_csv('poop.csv', index=False)
selected_columns.shape[0]

C:\Users\Joel\AppData\Local\Temp\ipykernel_14680\3990120306.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['TRANSACTION_DT'] = selected_columns['TRANSACTION_DT'].astype(str)


402720

In [ ]:
# remove rows with out properly formated dates
selected_columns = selected_columns[selected_columns['TRANSACTION_DT'].apply(len).isin([9, 10])]
selected_columns.shape[0]

402720

In [ ]:
# formate date column
selected_columns['Year'] = selected_columns['TRANSACTION_DT'].str[-6:-2]
selected_columns['Date'] = selected_columns['TRANSACTION_DT'].str[-8:-6]
selected_columns['Month'] = selected_columns['TRANSACTION_DT'].str[:-8]


In [ ]:
# remove rows that arnt within two years of the election
selected_columns = selected_columns[selected_columns['Year'].str.startswith((yeara, yearb))]
selected_columns.shape[0]

402223

## Verify Data Quality

In [ ]:
unique_combos = selected_columns[['Month', 'Year']]

# Group by 'Month' and 'Year', then get the counts
combo_counts = unique_combos.groupby(['Month', 'Year']).size().reset_index(name='Count')
combo_counts = combo_counts.apply(pd.to_numeric)
combo_counts = combo_counts.sort_values(by=['Year', 'Month'])
print(combo_counts)

   Month  Year   Count
0      1  2019   12588
2      2  2019   13663
3      3  2019   17221
4      4  2019   16285
6      5  2019   19484
8      6  2019    2693
5      4  2020   93943
7      5  2020   69535
1     11  2020  156811


In [ ]:
# Convert 'Count' column to list
count_list = combo_counts['Count'].tolist()

# Check if any value is less than 100
if any(count < 50 for count in count_list):
    print('WELP')

## A for loop for the 14 small file (same code as above)

In [ ]:
# create 'months' folder if it doesn't exist
if not os.path.exists('2020_files'):
    os.makedirs('2020_files')

In [ ]:
selected_columns.to_csv(f'2020_files/df{number}.csv', index=False)

In [ ]:
# Create a list of numbers from 001 to 014
number_list = [str(i).zfill(3) for i in range(4, 5)]

In [ ]:
number_list

['004']

In [ ]:
for n in number_list:
    #df1 = pd.read_csv('20.txt', sep='|', names=column_names)
    df1 = pd.read_csv(f'chunks/20.txt.{n}.txt', sep='|', engine='python', names=column_names, error_bad_lines=False)


    df1.shape[0]
    df1.head()

    ## Filter out irrelevant contribution data

    can_ids = pd.read_excel('filtered_project_committees.xlsx')
    can_ids_list = can_ids['committee_id'].to_list()

    len(can_ids_list)

    filtered_df1 = df1[df1['CMTE_ID'].isin(can_ids_list)]
    #filtered_df1.to_csv('test2_year_16.csv', index=False)
    filtered_df1.shape[0]

    ## Drop irrelevant columns and format date columns

    selected_columns = filtered_df1[['CMTE_ID', 'TRANSACTION_DT', 'TRANSACTION_AMT']]
    selected_columns['TRANSACTION_DT'] = selected_columns['TRANSACTION_DT'].astype(str)

    # remove rows with out properly formated dates
    selected_columns = selected_columns[selected_columns['TRANSACTION_DT'].apply(len).isin([9, 10])]

    # formate date column
    selected_columns['Year'] = selected_columns['TRANSACTION_DT'].str[-6:-2]
    selected_columns['Date'] = selected_columns['TRANSACTION_DT'].str[-8:-6]
    selected_columns['Month'] = selected_columns['TRANSACTION_DT'].str[:-8]

    # remove rows that arnt within two years of the election
    selected_columns = selected_columns[selected_columns['Year'].str.startswith((yeara, yearb))]
    selected_columns.shape[0]

    ## Verify Data Quality

    unique_combos = selected_columns[['Month', 'Year']]

    # Group by 'Month' and 'Year', then get the counts
    combo_counts = unique_combos.groupby(['Month', 'Year']).size().reset_index(name='Count')
    combo_counts = combo_counts.apply(pd.to_numeric)
    combo_counts = combo_counts.sort_values(by=['Year', 'Month'])
    print(combo_counts)

    # Convert 'Count' column to list
    count_list = combo_counts['Count'].tolist()

    # Check if any value is less than 100
    if any(count < 50 for count in count_list):
        print('WELP')
        break


    ## Create and download an excel file for each month of contributions

    # create 'months' folder if it doesn't exist
    if not os.path.exists('2020_files'):
        os.makedirs('2020_files')

    selected_columns.to_csv(f'2020_files/df{n}.csv', index=False)

C:\Users\Joel\AppData\Local\Temp\ipykernel_14680\4014223319.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df1 = pd.read_csv(f'chunks/20.txt.{n}.txt', sep='|', engine='python', names=column_names, error_bad_lines=False)
Skipping line 544571: '|' expected after '"'
Skipping line 781538: '|' expected after '"'
Skipping line 1448751: '|' expected after '"'
Skipping line 2557942: '|' expected after '"'
Skipping line 5079060: '|' expected after '"'


Empty DataFrame
Columns: [Month, Year, Count]
Index: []


C:\Users\Joel\AppData\Local\Temp\ipykernel_14680\4014223319.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['TRANSACTION_DT'] = selected_columns['TRANSACTION_DT'].astype(str)


## Create and download an excel file for each month of contributions

In [ ]:
folder_path = '2020_files/'

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each file in the folder
for i in range(1, 15):
    # Generate the file name
    filename = f'df{i:03d}.csv'
    filepath = os.path.join(folder_path, filename)

    # Read the CSV file into a DataFrame
    df = pd.read_csv(filepath)

    # Append the DataFrame to the list
    dfs.append(df)

In [ ]:
all_14 = pd.concat(dfs, ignore_index=True)
all_14.shape[0]

5114995

In [ ]:
#create a df with all 14 named all_14

In [ ]:
# create 'months' folder if it doesn't exist
if not os.path.exists('months'):
    os.makedirs('months')

In [ ]:
# list of all months
unique_year_month_combos = all_14[['Year', 'Month']].drop_duplicates()
year_month_list = [(row['Year'], row['Month']) for _, row in unique_year_month_combos.iterrows()]

# iterate over each month
for year, month in year_month_list:
    print(month, year)
    filtered_df = all_14[(all_14['Year'] == year) & (all_14['Month'] == month)]

    # save to the folder
    file_path = os.path.join('months/', f'{year}_{month}.csv')
    filtered_df.to_csv(file_path, index=False)

print("Data frames have been saved successfully.")

9 2020
10 2020
6 2020
8 2020
11 2019
11 2020
10 2019
3 2020
4 2020
12 2020
1 2020
12 2019
7 2019
6 2019
4 2019
5 2019
2 2019
1 2019
3 2019
5 2020
9 2019
8 2019
7 2020
2 2020
Data frames have been saved successfully.
